In [15]:
import os
import numpy as np
import cv2
import time
import imutils
include <opencv2/core/types.hpp>

SyntaxError: invalid syntax (<ipython-input-15-b3d059385758>, line 6)

In [41]:
video_names = ['airport.mp4','car_chase_01.mp4','car_chase_02.mp4','overpass.mp4']
video_directory = '/Users/sanjaymsanthosh/Desktop/projects/project 2019/human detection/yolo/yolo-object-detection/videos'
video_files = os.path.sep.join([video_directory,video_names[1]])
output = '/Users/sanjaymsanthosh/Desktop/projects/project 2019/human detection/yolo/yolo-object-detection/output/sample.mjpg'
yolo_directory = "/Users/sanjaymsanthosh/Desktop/projects/project 2019/human detection/yolo/yolo-object-detection/yolo-coco"

In [3]:
label_path = os.path.sep.join([yolo_directory,'coco.names'])
Labels = open(label_path).read().split('\n')

In [4]:
np.random.seed(42)
Colors = np.random.randint(0,255,[len(Labels),3])

In [5]:
weightsPath = os.path.sep.join([yolo_directory, "yolov3.weights"])
configPath = os.path.sep.join([yolo_directory, "yolov3.cfg"])

print('[INFO] loading yolo pre trained from disk')
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading yolo pre trained from disk


In [6]:
vs = cv2.VideoCapture(video_files)
writer = None
(W, H) = (None, None)

try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1



[INFO] 350 total frames in video


In [61]:
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    if not grabbed:break
        
    if W is None or H is None:
        (H, W) = frame.shape[:2]


    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > .5:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply nonmaxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, .5,
        .3)

    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in Colors[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(Labels[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # check if the video writer is None
    
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        #writer = cv2.VideoWriter(output, fourcc, 30,
        #s   size,True)
        
        print(frame_width)
        print(type(frame))
        size = np.array([720],ndmin=1)
        size = np.array((np.asscalar(np.array(720)),np.asscalar(np.array(720))))
        writer = cv2.VideoWriter(output,cv2.VideoWriter_fourcc('M','J','P','G'), 10, size,True)
        writer = cv2.VideoWriter()

        # some information on processing single frame
        if total > 0:
            elap = (end - start)
            print("[INFO] single frame took {:.4f} seconds".format(elap))
            print("[INFO] estimated total time to finish: {:.4f}".format(
                elap * total))

    # write the output frame to disk
    writer.write(frame)



720
<class 'numpy.ndarray'>


TypeError: only size-1 arrays can be converted to Python scalars

In [23]:
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...


AttributeError: 'NoneType' object has no attribute 'release'